# UCLQ Quantum Energy Hackathon - JASTRZĘBIE TEAM



### 1.   Install Qiskit



In [1]:
!pip install qiskit

     |████████████████████████████████| 6.5 MB 29.7 MB/s 
     |████████████████████████████████| 18.0 MB 101 kB/s 
     |████████████████████████████████| 238 kB 53.9 MB/s 
     |████████████████████████████████| 200 kB 48.3 MB/s 
     |████████████████████████████████| 54 kB 2.5 MB/s 
     |████████████████████████████████| 1.6 MB 36.9 MB/s 
     |████████████████████████████████| 38.1 MB 1.2 MB/s 
     |████████████████████████████████| 49 kB 5.9 MB/s 
     |████████████████████████████████| 37.5 MB 421 kB/s 
     |████████████████████████████████| 943 kB 50.2 MB/s 
     |████████████████████████████████| 49 kB 6.4 MB/s 
     |████████████████████████████████| 3.6 MB 43.1 MB/s 
     |████████████████████████████████| 113 kB 50.3 MB/s 
  Created wheel for qiskit: filename=qiskit-0.34.2-py3-none-any.whl size=11805 sha256=255b3abd23ee1b97654ffeed87623003938092cd164cfe262675a5cc4f973bc6
  Stored in directory: /root/.cache/pip/wheels/62/77/65/cda6eedfdd2a525bd3f479a4386930ae3088a1eb01f8c

In [2]:
%matplotlib inline
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import Aer, execute
from qiskit.circuit import Parameter
from qiskit.visualization import plot_histogram
from scipy.optimize import minimize


### 2. Define parameters

In [ ]:
# Renewable sources: solar, on-shore and off-shore wind turbines
number_of_renewable_sources = 3
# Locations: England, Northern Ireland, Scotland, Wales
number_of_locations = 4

# Calculate the number of qubits needed, 
# NOTE: IBMQ systems accesible have 5 qubits
nqubits = number_of_locations * number_of_renewable_sources

G = nx.complete_graph(nqubits)

total_cost = C = 61510030000.0 # units: £

# Calculate revenue for each energy source in all locations 
# dict{location: [energy_source]}
location_revenue_per_unit = {
    0: [2783.55, 336113.74, 153105454.50],
    1: [2332.31, 434451.22, 0.0],
    2: [1088.38, 1020438.50, 85798571.43],
    3: [3573.59, 768855.93, 140790000],
}

# Calculate cost for each energy source in all locations
# dict{location: [energy_source]}
location_cost_per_unit = {
    0: [300.92, 52370.72, 22696783.84],
    1: [273.66, 70226.14, 0.0],
    2: [116.17, 161395.70, 11626961.90],
    3: [382.92, 122708.47, 21872844.44],
}

# variable and a rough estimate, see spreadsheet for current figures.
# dict{location: [energy_source]}
location_unit_capacity = {
    0: [800000, 5000, 40],
    1: [30000, 1400, 0],
    2: [70000, 4000, 10],
    3: [60000, 1000, 5],
}

# lambda and gamma, the initial parameter guess for the cost function
lam, gam = 1.0, 1.0

# Calculate the total costs and revenues 
a_revs, b_costs = [], []

for N in range(number_of_locations):
    for R in range(number_of_renewable_sources):
        num_units = location_unit_capacity[N][R]
        a_revs.append(num_units * location_revenue_per_unit[N][R])
        b_costs.append(num_units * location_cost_per_unit[N][R])
        

### 3. The cost function